In [ ]:
import math
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import numpy as np

In [ ]:
import pandas as pd

In [ ]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder

In [ ]:
from torch.distributions.bernoulli import Bernoulli

In [ ]:
df = pd.read_csv('mushrooms.csv').to_numpy()

In [ ]:
y = df[:,0:1]
x = df[:,1:]

In [ ]:
print(x.shape,y.shape)

In [ ]:
x_enc = OneHotEncoder(handle_unknown='ignore')
y_enc = LabelEncoder()
x_encoded = x_enc.fit_transform(x).toarray()
y_encoded = np.expand_dims(y_enc.fit_transform(y),1)

In [ ]:
print(x_encoded.shape,y_encoded.shape)

In [ ]:
#better to randomly pick the test sample
x_train = torch.split(torch.tensor(x_encoded[:4062,:]).float(),16)
y_train = torch.split(torch.tensor(y_encoded[:4062,:]).float(),16)
x_test = torch.split(torch.tensor(x_encoded[4062:,:]).float(),16)
y_test = torch.split(torch.tensor(y_encoded[4062:,:]).float(),16)
#print(x_train.shape, y_train.shape,x_test.shape,y_test.shape)

In [ ]:
class AutoTuning(nn.Module):

    def __init__(self): #, dropout=0.1
        super(AutoTuning, self).__init__()
        self.layer1 = nn.parameter.Parameter(torch.empty((117,1000),requires_grad=True))
        torch.nn.init.xavier_uniform_(self.layer1,gain=nn.init.calculate_gain('relu'))
        self.b1 = nn.parameter.Parameter(torch.zeros((1, 1000), requires_grad=True))
        self.layer2 = nn.parameter.Parameter(torch.empty((1000,1),requires_grad=True))
        torch.nn.init.xavier_uniform_(self.layer2,gain=nn.init.calculate_gain('sigmoid'))
        self.b2 = nn.parameter.Parameter(torch.zeros((1),requires_grad=True))
        
        self.parameter_matrix = torch.empty((117,1000),requires_grad=True)
        torch.nn.init.xavier_uniform_(self.parameter_matrix,gain=nn.init.calculate_gain('relu'))
        print(torch.max(self.parameter_matrix))
        self.parameter_matrix = nn.parameter.Parameter(self.parameter_matrix - 0.1035) #-0.1035, 4
        
    def forward(self,x,x_test,print_stat):
        binary_params = torch.tanh(self.parameter_matrix) + (torch.relu(torch.sign(self.parameter_matrix)) - torch.tanh(self.parameter_matrix)).detach()
        #binary_params = torch.sign(self.parameter_matrix)
        #binary_params = Bernoulli(torch.sigmoid(self.parameter_matrix)).sample().requires_grad_()
        out1 = torch.relu(torch.mm(x,self.layer1*binary_params.detach()) + self.b1)
        out2 = torch.sigmoid(torch.mm(out1,self.layer2) + self.b2)
        
        test1 = torch.relu(torch.mm(x_test,self.layer1.detach()*binary_params) + self.b1.detach())
        test2 = torch.sigmoid(torch.mm(test1,self.layer2.detach()) + self.b2.detach())
        if print_stat:
            with torch.no_grad():
                print(torch.sum(binary_params))
        return out2,test2

In [ ]:
model = AutoTuning()
bce_loss = nn.BCELoss()
lr = 0.5 # learning rate
optimizer = torch.optim.SGD(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.95)
test_prec = 0 
prec_counter = 0
prec_limit = 3
for epoch in range(10):
    for i,x_batch in enumerate(x_train):
        optimizer.zero_grad()
        out,out_test = model(x_batch,x_test[i],True)
        loss = bce_loss(out,y_train[i])
        loss_test = bce_loss(out_test,y_test[i])
        print(loss.item(),loss_test.item())
        loss.backward() #retain_graph=True
        loss_test.backward()
        optimizer.step()
    #new_test_prec = torch.sum((model(x_train,x_test,False)[1]>0.5).int()==y_test)/len(y_test)
    #print(epoch,loss.item(),new_test_prec)
    #if new_test_prec < test_prec:
    #    prec_counter += 1
    #    if prec_counter > prec_limit:
    #        #break
    #        pass
    #else:
        test_prec = new_test_prec


In [ ]:
y_test

In [ ]:
torch.nn.Softmax

In [ ]:
t1 = torch.randn((20,10))
t2 = torch.randn((10,2))
b1 = torch.ones((1,2))
torch.mm(t1,t2) + b1

In [ ]:
torch.tanh(torch.tensor(5.5))

In [ ]:
layer1 = nn.parameter.Parameter(torch.empty((117,100)),requires_grad = True)
torch.nn.init.xavier_uniform_(layer1,gain=nn.init.calculate_gain('relu'))
#layer1 = layer1 - 0.
print(torch.sum(torch.relu(torch.sign(layer1))))
torch.sum(torch.relu(layer1))
test_copy = layer1.detach().clone()
layer1 * test_copy

In [ ]:
layer1

In [ ]:
test_binary = Bernoulli(torch.sigmoid(torch.randn((10,10)))).sample()

In [ ]:
test_binary

In [ ]:
torch.exp(torch.randn((10,10)))